# Monitoring setup for Bicycle Sharing Demand Prediction

This notebook shows how you can use the Evidently to:
* calculate prerformance and data drift for the model, performed as batch checks 
* log models quality & data drift using MLflow Tracking
* explore the result 

More examples are avaliable in the github: https://github.com/evidentlyai/evidently/tree/main/examples

Evidently docs: https://docs.evidentlyai.com/

Join our Discord: https://discord.com/invite/xZjKRaNp8b

In [1]:
import datetime
import pandas as pd
import numpy as np
import requests
import zipfile
import io
import json

from sklearn import datasets, ensemble, model_selection
from scipy.stats import anderson_ksamp

from evidently.metrics import RegressionQualityMetric, RegressionErrorPlot, RegressionErrorDistribution
from evidently.metric_preset import DataDriftPreset, RegressionPreset
from evidently.pipeline.column_mapping import ColumnMapping
from evidently.report import Report

/Users/mnrozhkov/dev/mlrepa/com/evidently/evidently-mlflow/.venv/lib/python3.10/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Users/mnrozhkov/dev/mlrepa/com/evidently/evidently-mlflow/.venv/lib/python3.10/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Users/mnr

In [2]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

## Bicycle Demand Data

More information about the dataset can be found in UCI machine learning repository: https://archive.ics.uci.edu/ml/datasets/bike+sharing+dataset

Acknowledgement: Fanaee-T, Hadi, and Gama, Joao, 'Event labeling combining ensemble detectors and background knowledge', Progress in Artificial Intelligence (2013): pp. 1-15, Springer Berlin Heidelberg

In [3]:
content = requests.get("https://archive.ics.uci.edu/static/public/275/bike+sharing+dataset.zip").content
with zipfile.ZipFile(io.BytesIO(content)) as arc:
    raw_data = pd.read_csv(arc.open("hour.csv"), header=0, sep=',', parse_dates=['dteday']) 

In [4]:
raw_data.index = raw_data.apply(lambda row: datetime.datetime.combine(row.dteday.date(), datetime.time(row.hr)),
                                axis=1)

In [5]:
raw_data.head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
2011-01-01 00:00:00,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
2011-01-01 01:00:00,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2011-01-01 02:00:00,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
2011-01-01 03:00:00,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
2011-01-01 04:00:00,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


In [6]:
target = 'cnt'
prediction = 'prediction'
numerical_features = ['temp', 'atemp', 'hum', 'windspeed', 'mnth', 'hr', 'weekday']
categorical_features = ['season', 'holiday', 'workingday', ]#'weathersit']

In [7]:
reference = raw_data.loc['2011-01-01 00:00:00':'2011-01-28 23:00:00']
current = raw_data.loc['2011-01-29 00:00:00':'2011-02-28 23:00:00']

In [8]:
# X_train, X_test, y_train, y_test = model_selection.train_test_split(
#     reference[numerical_features + categorical_features],
#     reference[target],
#     test_size=0.3
# )

## Load the Model (from the MLFlow)

In [9]:
regressor = ensemble.RandomForestRegressor(random_state = 0, n_estimators = 50)

In [12]:
preds_train = regressor.predict(X_train)
preds_test = regressor.predict(X_test)

## Production model training

In [16]:
regressor.fit(reference[numerical_features + categorical_features], reference[target])

RandomForestRegressor(n_estimators=50, random_state=0)

In [17]:
column_mapping = ColumnMapping()

column_mapping.target = target
column_mapping.prediction = prediction
column_mapping.numerical_features = numerical_features
column_mapping.categorical_features = categorical_features

In [18]:
ref_prediction = regressor.predict(reference[numerical_features + categorical_features])
reference['prediction'] = ref_prediction

In [40]:
regression_performance_report = Report(metrics=[
    RegressionPreset(),
])

regression_performance_report.run(reference_data=None, current_data=reference,
                                  column_mapping=column_mapping)
# regression_performance_report

{'metrics': [{'metric': 'RegressionQualityMetric',
   'result': {'columns': {'utility_columns': {'date': None,
      'id': None,
      'target': 'cnt',
      'prediction': 'prediction'},
     'num_feature_names': ['atemp',
      'hr',
      'hum',
      'mnth',
      'temp',
      'weekday',
      'windspeed'],
     'cat_feature_names': ['season', 'holiday', 'workingday'],
     'text_feature_names': [],
     'datetime_feature_names': ['dteday'],
     'target_names': None},
    'current': {'r2_score': 0.9834769538243223,
     'rmse': 40.00879093851133,
     'mean_error': -0.006019417475728246,
     'mean_abs_error': 4.105695792880258,
     'mean_abs_perc_error': 16.196696009106397,
     'abs_error_max': 50.24000000000001,
     'underperformance': {'majority': {'mean_error': 0.09179856115107903,
       'std_error': 3.871470563870639},
      'underestimation': {'mean_error': -15.78451612903226,
       'std_error': 7.2438087765620836},
      'overestimation': {'mean_error': 14.018064516129

# Model Performance

## Some time has passed - how is the model working?

In [20]:
current_prediction = regressor.predict(current[numerical_features + categorical_features])
current['prediction'] = current_prediction

### Week 1

In [42]:
regression_performance_report.run(reference_data=reference, current_data=current.loc['2011-01-29 00:00:00':'2011-02-07 23:00:00'],
                                  column_mapping=column_mapping)
# regression_performance_report

### Week 2

In [43]:
regression_performance_report = Report(metrics=[
    RegressionQualityMetric(),
    RegressionErrorPlot(),
    RegressionErrorDistribution()
])

regression_performance_report.run(reference_data=reference, current_data=current.loc['2011-02-07 00:00:00':'2011-02-14 23:00:00'], 
                                            column_mapping=column_mapping)
# regression_performance_report.show()

### Week 3

In [44]:
regression_performance_report = Report(metrics=[
    RegressionQualityMetric(),
    RegressionErrorPlot(),
    RegressionErrorDistribution()
])

regression_performance_report.run(reference_data=reference, current_data=current.loc['2011-02-15 00:00:00':'2011-02-21 23:00:00'], 
                                            column_mapping=column_mapping)
# regression_performance_report.show()

## Why the quality has dropped?

In [24]:
column_mapping_drift = ColumnMapping()

column_mapping_drift.target = target
column_mapping_drift.prediction = prediction
column_mapping_drift.numerical_features = numerical_features
column_mapping_drift.categorical_features = []

In [46]:
# data_drift_report = Report(metrics=[
#     DataDriftPreset(),
# ])

# data_drift_report.run(
#     reference_data=reference,
#     current_data=current.loc['2011-02-14 00:00:00':'2011-02-21 23:00:00'],
#     column_mapping=column_mapping_drift,
# )
# # data_drift_report

# Model Quality Evaluation (Prod)

In [48]:
import mlflow
#import mlflow.sklearn
from mlflow.tracking import MlflowClient

In [49]:
experiment_batches = [
    ('2011-01-29 00:00:00','2011-02-07 23:00:00'),
    ('2011-02-07 00:00:00','2011-02-14 23:00:00'),
    ('2011-02-15 00:00:00','2011-02-21 23:00:00'),  
]

In [50]:
the_report = Report(metrics=[
    RegressionQualityMetric(),
    RegressionErrorPlot(),
    RegressionErrorDistribution(),
])

the_report.run(
    reference_data=reference, 
    current_data=current.loc[experiment_batches[0][0]:experiment_batches[0][1]],
    column_mapping=column_mapping_drift
)

In [52]:
# the_report.as_dict()

In [53]:
logged_json = json.loads(the_report.json())

In [54]:
logged_json

{'version': '0.3.2.1',
 'timestamp': '2023-07-04 18:48:50.559089',
 'metrics': [{'metric': 'RegressionQualityMetric',
   'result': {'columns': {'utility_columns': {'date': None,
      'id': None,
      'target': 'cnt',
      'prediction': 'prediction'},
     'num_feature_names': ['atemp',
      'hr',
      'hum',
      'mnth',
      'temp',
      'weekday',
      'windspeed'],
     'cat_feature_names': [],
     'text_feature_names': [],
     'datetime_feature_names': ['dteday'],
     'target_names': None},
    'current': {'r2_score': 0.8342907435814624,
     'rmse': 442.6668527659575,
     'mean_error': -6.26536170212766,
     'mean_abs_error': 13.52936170212766,
     'mean_abs_perc_error': 41.57714593080225,
     'abs_error_max': 94.34,
     'underperformance': {'majority': {'mean_error': -4.857251184834124,
       'std_error': 12.75642799788315},
      'underestimation': {'mean_error': -65.01666666666665,
       'std_error': 19.82673462929804},
      'overestimation': {'mean_error': 

In [55]:
[x['metric'] for x in logged_json['metrics']]

['RegressionQualityMetric',
 'RegressionErrorPlot',
 'RegressionErrorDistribution']

In [56]:
logged_json['metrics'][0]['result']['current']['mean_error']

-6.26536170212766

In [63]:
from config import MLFLOW_TRACKING_URI


mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
client = MlflowClient()
print(f"Client tracking uri: {client.tracking_uri}")

#set experiment
mlflow.set_experiment('Model Quality Evaluation')

#start new run
for date in experiment_batches:
    with mlflow.start_run() as run: #inside brackets run_name='test'
        
        # Log parameters
        mlflow.log_param("begin", date[0])
        mlflow.log_param("end", date[1])

        # Get metrics
        the_report = Report(metrics=[
            RegressionQualityMetric(),
            RegressionErrorPlot(),
            RegressionErrorDistribution(),
        ])
        the_report.run(
            reference_data=reference, 
            current_data=current.loc[date[0]:date[1]],
            column_mapping=column_mapping_drift)
        logged_json = json.loads(the_report.json())
        
        me = logged_json['metrics'][0]['result']['current']['mean_error']
        mae = logged_json['metrics'][0]['result']['current']["mean_abs_error"]
        
        # Log metrics
        mlflow.log_metric('me', round(me, 3))
        mlflow.log_metric('mae', round(mae, 3))

        print(run.info)

2023/07/04 18:50:33 INFO mlflow.tracking.fluent: Experiment with name 'Model Quality Evaluation' does not exist. Creating a new experiment.


Client tracking uri: http://localhost:5000
<RunInfo: artifact_uri='mlflow-artifacts:/202220522012713527/f32555b490344e6ea9071972ad94fc7c/artifacts', end_time=None, experiment_id='202220522012713527', lifecycle_stage='active', run_id='f32555b490344e6ea9071972ad94fc7c', run_name='mysterious-cod-126', run_uuid='f32555b490344e6ea9071972ad94fc7c', start_time=1688489433952, status='RUNNING', user_id='mnrozhkov'>
<RunInfo: artifact_uri='mlflow-artifacts:/202220522012713527/7778dfdafc5e4195a405803c9f824436/artifacts', end_time=None, experiment_id='202220522012713527', lifecycle_stage='active', run_id='7778dfdafc5e4195a405803c9f824436', run_name='bemused-kit-919', run_uuid='7778dfdafc5e4195a405803c9f824436', start_time=1688489434209, status='RUNNING', user_id='mnrozhkov'>
<RunInfo: artifact_uri='mlflow-artifacts:/202220522012713527/9e604c3211b0468d851c1e98b67a71ff/artifacts', end_time=None, experiment_id='202220522012713527', lifecycle_stage='active', run_id='9e604c3211b0468d851c1e98b67a71ff', 